Most of the code, as well as the resulting folder structures, is based on https://cs230-stanford.github.io/pytorch-getting-started.html and 
https://github.com/cs230-stanford/cs230-code-examples/tree/master/pytorch/vision.

# 1. Dataset Preprocessing
##### Required Folder Structure:
- data
 - data/heart_scans *(directory for "data_dir")*
 
##### Parameters:
 - data_dir: (string) The location of the original dataset. May contain subfolders.
 - output_dir: (string) The location where the preprocessed dataset should be stored.
 - n4: (bool) Whether N4 Bias Field Correction is applied to the dataset.
 - roi: (bool) Whether rudimentary ROI is applied to the dataset.
 - rot: (bool) Whether dataset is augmented by rotating it 3 times in semi-random degrees.
 - h_flip: (bool) Whether dataset is augmented by horizontally flipping it.
 - v_flip: (bool) Whether dataset is augmented by vertically flipping it.
 - scale: (bool) Wheter dataset is augemented by random scaling.

In [1]:
import build_dataset

build_dataset.main(data_dir="data/heart_scans", 
                   output_dir="data/{}x{}_heart_scans".format(build_dataset.IMG_SIZE, build_dataset.IMG_SIZE,), 
                   n4=False, 
                   roi=False, 
                   rot=True, 
                   h_flip=False, 
                   v_flip=False, 
                   scale=False)

Checking for missing ENDO or EPI ...
   Done. Found 0 missing ENDO and 0 missing EPI.
Split filenames into train and test set ...
    Done.
Resizing train data, saving resized data to data\320x320_heart_scans_rot\train_heart_scans


100%|██████████| 840/840 [00:10<00:00, 54.68it/s] 


Augmenting train data, saving augmented data to data\320x320_heart_scans_rot\train_heart_scans


100%|██████████| 840/840 [00:32<00:00, 10.58it/s]  


Resizing test data, saving resized data to data\320x320_heart_scans_rot\test_heart_scans


100%|██████████| 171/171 [00:02<00:00, 85.26it/s]


Augmenting test data, saving augmented data to data\320x320_heart_scans_rot\test_heart_scans


100%|██████████| 171/171 [00:07<00:00, 16.18it/s]


Done building dataset.


# 2. Experiments using k-fold CV (Phase 1)
## 2.1. Single Hyperparameter Search
##### Required Folder Structure:
- data
 - data/320x320_heart_scans<postfix indicating augmentation, n4, roi\> *(directory for "data_dir")*
    - data/320x320_heart_scans<postfix indicating augmentation, n4, roi\>/train_heart_scans
    - data/320x320_heart_scans<postfix indicating augmentation, n4, roi\>/test_heart_scans
- experiments
 - ... *(directory for "model_dir" and "parent_dir"; must contain hyper_params.json, as specified below)*

##### Required JSON File:
Must contain a file named "hyper_params.json" with the following content \[possible values specified after \>\]:

`{
    "endo_or_epi": "EPI",     > ["EPI", "ENDO"]
    "model": "ModelA",        > ["ModelA", "UNetOriginal", "UNetOriginalPretrained"]
    "augmentation": "_rot",   > ["_rot", "_rot_flip_scale", "n4_rot", ...]
    "learning_rate": 0.0004, 
    "optimizer": "Adam",      
    "loss": "soft_dice_loss", > ["soft_dice_loss", "binary_cross_entropy_loss"]
    "activation": "ReLU",     > ["ReLU", "ELU"]
    "batch_size": 12,
    "num_epochs": 45,
    "batch_norm": 1,          > [0, 1]
    "dropout_rate": 0.0,
    "weight_decay": 0.0,
    "treshold": 0.5,
    "num_workers": 4
}`

##### Parameters:
- data_dir: (string) The location of the preprocessed dataset, as was specified as "output_dir" during "1. Dataset Preprocessing", **_not including the postfix indicating the augmentation or application of N4 and ROI_**!
- model_dir: (string) Directory containing hyper_params.json and/or, for transfer learning, the saved model as a \*.pth.tar file.
- restore_file: (string) If transfer learning is used, specify name of the \*.pth.tar file containing the starting weights and biases, omitting ".pth.tar". Must be located in the location of "model_dir". None, for no transfer learning (random weight initialization).
- k_folds: (int) Specify the number of folds (k) for k-fold CV. (Setting to 0 to turn off k-fold CV will result in "3. Final Training of Models (Phase 2)" taking place.)

In [1]:
import train

# Example: Perform hyperparameter experiment for endocardium for a batch size of 12.
train.main(data_dir="data/320x320_heart_scans", 
           model_dir="experiments/ENDO/ModelA_random/batch_size/batch_size_12", 
           restore_file=None, 
           k_folds=5)

added new handler!


Loading the datasets...


Mean: 300.0052185058594, Std: 430.4715881347656


- done.
For k-fold 1/5:
Starting training for 45 epoch(s)
Epoch 1/45
100%|██████████| 75/75 [00:28<00:00,  3.43it/s, loss=0.959]
- Train metrics: dsc: 0.274 ; iou: 0.182 ; accuracy: 0.920 ; precision: 0.182 ; recall: 0.928 ; loss: 0.959
- Eval metrics : dsc: 0.484 ; iou: 0.345 ; accuracy: 0.985 ; precision: 0.345 ; recall: 0.968 ; loss: 0.950


Checkpoint Directory exists! 


- Found new best DSC
Epoch 2/45
 51%|█████     | 38/75 [00:15<00:12,  3.07it/s, loss=0.943]


KeyboardInterrupt: 

In [ ]:
import train

# Same example for Cross Transfer Learning.
train.main(data_dir="data/320x320_heart_scans", 
           model_dir="experiments/ENDO/ModelA_pretrained/batch_size/batch_size_12", 
           restore_file="best_epi_exp_68", 
           k_folds=5)

## 2.2. Multiple Hyperparameter Search
##### Required Folder Structure:
Same as for "2.1. Single Hyperparameter Search".
##### Required JSON File:
Same as for "2.1. Single Hyperparameter Search", but requires that an array is specified for exactly one of the hyperparameters, e.g.:

`{
    ...    
    "learning_rate": [0.01, 0.001, 0.0001], 
    ...
}`

##### Parameters:
- data_dir: (string) The location of the preprocessed dataset, as was specified as "output_dir" during "1. Dataset Preprocessing", **_not including the postfix indicating the augmentations or application of N4 and ROI_**!
- parent_dir: (string) Folder containing hyper_params.json, specifying the different values for the hyperparameter to be tried.

In [ ]:
import search_hyperparams

# Example: Perform hyperparameter search for epicardium over various learning rates.
search_hyperparams.main(data_dir="data/320x320_heart_scans",
                        parent_dir="experiments/EPI/learning_rate")

## 2.3. Numerical Evaluation of Experiments
##### Required Folder Structure:
- experiments *(directory for "parent_dir")*
 - ... *(subfolders must contain various JSON files, as specified below)*
 
##### Required JSON Files:
All the JSON files that were generated by "2. Experiments using k-fold CV (Phase 1)", e.g. "metrics_k_fold_train.json" or "metrics_k_fold_val_average_best.json". Must be located in the directory specified for "parent_dir" or in any of its subfolders.

##### Parameters:
- parent_dir: (string) Folder containing (subfolders with) the JSON files created by all experiments run for "2. Experiments using k-fold CV (Phase 1)". The file "results.md", containing the tabular overview generated by running "synthesize_results", will be located here.

In [ ]:
import synthesize_results

synthesize_results.main(parent_dir="experiments")

## 2.4. Graphical Evaluation of Experiments
For a graphical evaluation of experiments, start a console at the same directory containing this file and run "tensorboard --logdir=tensor_log\\_<specifiy subfolder structure for the graphs you want to evaluate\>_", then follow instructions in the console.

# 3. Final Training of Models (Phase 2)
##### Required Folder Structure:
Same as for "2. Experiments using k-fold CV (Phase 1)".
##### Required JSON File:
Same as for "2.1. Single Hyperparameter Search".
##### Parameters:
- data_dir: (string) The location of the preprocessed dataset, as was specified as "output_dir" during "1. Dataset Preprocessing", **_not including the postfix indicating the augmentation or application of N4 and ROI_**!
- model_dir: (string) Folder containing hyper_params.json and/or, for transfer learning, the saved model as a \*.pth.tar file.
- restore_file: (string) If transfer learning is used, specify name of the \*.pth.tar file containing the starting weights and biases, omitting ".pth.tar". Must be located in the location of "model_dir". None, for no transfer learning (random weight initialization).
- k_folds: (int) Set to 0 to turn off k-fold CV for this phase.

In [ ]:
import train

# Example: Final Training for endocardium, with batch size 12.
train.main(data_dir="data/320x320_heart_scans", 
           model_dir="experiments/ENDO/ModelA_random/batch_size/batch_size_12", 
           restore_file=None, 
           k_folds=0)

## 3.1. Optional: Run Numerical Evaluation again (see 2.3.)

In [ ]:
import synthesize_results

synthesize_results.main(parent_dir="experiments")

# 4. Segmenting LV Images
##### Required Folder Structure:
- data
 - data/320x320_heart_scans<postfix indicating augmentation, n4, roi\> *(directory for "data_dir")*
    - data/320x320_heart_scans<postfix indicating augmentation, n4, roi\>/test_heart_scans *(containing the preprocessed images to be segmented)*
- experiments
 - ... *(directory for "model_dir"; must contain hyper_params.json, as specified below; must contain *.pth.tar file containing the trained model)*

##### Required JSON File:
- Same structure as for "2.1. Single Hyperparameter Search", with additional entries for "mean" and "bias".
- MUST be the one modified by "3. Final Training of Models (Phase 2)", so that it contains the correct mean and bias of the training set the model was trained with.

##### Parameters:
- data_dir: (string) The location of the preprocessed images to be segmented, as was specified as "output_dir" during "1. Dataset Preprocessing", **_not including the postfix indicating the augmentation or application of N4 and ROI_**!
- model_dir: (string) Folder containing hyper_params.json and the saved model as a \*.pth.tar file.
- restore_file: (string) Specify name of the \*.pth.tar file containing the model trained in "3. Final Training of Models (Phase 2)", omitting ".pth.tar". Must be located in the location of "model_dir".

In [ ]:
import evaluate

# Endocardium Segmentation using the model proposed as best, from Experiment 35 (see Thesis).
evaluate.main(data_dir="data/320x320_heart_scans", 
              model_dir="experiments/ENDO/best_exp_no_35", 
              restore_file="best_endo_exp_35")

In [ ]:
import evaluate

# Epicardium Segmentation using the model proposed as best, from Experiment 68 (see Thesis).
evaluate.main(data_dir="data/320x320_heart_scans", 
              model_dir="experiments/EPI/best_exp_no_68", 
              restore_file="best_epi_exp_68")

## 4.1. Optional: Run Numerical Evaluation again (see 2.3.)

In [ ]:
import synthesize_results

synthesize_results.main(parent_dir="experiments")

## 4.2. Optional: Numerical Results per Image

Look at "metrics_test_single_file_names.json" for the numerical results for each image, sorted from best to worst.

Example:

`{
    "LAED-BL-S_CineMR_ti00_sl07_EPI_AUG2.png": {
        "dsc": 0.9869700074195862,
        "iou": 0.9742752313613892,
        "accuracy": 0.999384765625,
        "precision": 0.9843234419822693,
        "recall": 0.9896308779716492,
        "loss": 0.01472163200378418
    },
    "BATH-BLn2_CineMR_ti00_sl06_EPI.png": {
        "dsc": 0.9865750670433044,
        "iou": 0.9735058546066284,
        "accuracy": 0.99916015625,
        "precision": 0.9884266257286072,
        "recall": 0.9847304224967957,
        "loss": 0.01432490348815918
    },
    "GOMA-BL-S_CineMR_ti00_sl06_EPI_AUG2.png": {
        "dsc": 0.9836491942405701,
        "iou": 0.9678245186805725,
        ...
    },
  ...
}`